In [ ]:
#yyyymmdd,pre vs post,test/control, order_type,marketplace_seg,FM,LM,APR,AOR,G2N ,Mapped_exp,cobra,ocara,

In [3]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [48]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        #host="presto.processing.yoda.run",
        #host ="prime-trino-2.serving.data.production.internal",
        host = "processing-2.processing.data.production.internal",
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [49]:
data=pd.DataFrame()

for i in pd.date_range('20230704','20230722'):
    
    query="""
    with order_filtering as 
    (
        select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,drop_location_hex_8,
                CASE 
                    WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                    WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                    WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                    WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                    ELSE 'rest_evening' END AS time_period, order_status,quarter_hour,

                case when order_status != 'dropped' then 0
                else 1 end as net_orders
                
                --case when order_status != 'dropped' then 'unfulfilled_order'
                --else 'fulfilled_order' end as fulfilment_flag

        from 
            orders.order_logs_snapshot
             --where yyyymmdd = '{dt}' --'20230710' 
            where yyyymmdd = cast({dt} as varchar)
            and service_obj_service_name = 'Auto'
            and lower(trim(city_name)) in ('hyderabad','pune') 
    ),

    rider_pings as

    (
        select yyyymmdd,data_orderid,
            count(distinct data_orderid) as Gross_orders,
            count(distinct case when event_type = 'rider_busy' then unique_ping_level_id end) as riderBusy_pings,
            count(distinct case when event_type = 'rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
            count(distinct case when event_type = 'rider_accepted' then unique_ping_level_id end) as accepted_pings,
            count(distinct case when event_type = 'order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
            count(distinct case when event_type = 'rider_accept_failed' then unique_ping_level_id end) as rider_accept_failure_pings,        
            count(distinct case when event_type = 'rider_acknowledged' then unique_ping_level_id end) as rider_ack_pings
        from
        (
               select yyyymmdd,data_propagationBatchId,data_riderId,data_orderid,
                event_type_agg,eventtype as event_type,
                unique_ping_level_id,unique_id_order_level_id

                from
                (
            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            array_agg(eventtype) over (partition by data_orderid) as event_type_agg,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
            from raw.kafka_dispatch_propagation_immutable
            where yyyymmdd = cast({dt} as varchar)
            --where yyyymmdd = '20230710'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_acknowledged')


            union all

            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            array_agg(eventtype) over (partition by data_orderid) as event_type_agg,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch

            from(
            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
            row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

            from raw.kafka_dispatch_propagation_immutable
            where yyyymmdd = cast({dt} as varchar)
             --where yyyymmdd = '20230710'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                ) base
            where row =1
                    )
                )


                group by 1,2--,3
    ),

    immutabl_raw as (

                           select yyyymmdd, city_name,order_id,captain_id,event_type,order_status,updated_epoch,accept_to_pickup_distance

                            from orders.order_logs_immutable
                                where yyyymmdd = cast({dt} as varchar) --'{dt}'
                                and service_obj_service_name in ('Auto')
                                and order_id in (select distinct order_id from order_filtering)
                                and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)

                 ),

immutable_v2   as 

            ( 
                select yyyymmdd, city_name,order_id,
                --count(distinct order_id) as gross_orders,
                count(distinct case when order_level_end_state='COBRA' then order_id end) as COBRA,
                count(distinct case when order_level_end_state='Mapped_expiry' then order_id end) as Mapped_expiry,
                count(distinct case when order_level_end_state='OCARA' then order_id end) as OCARA

                from
                (
                select yyyymmdd, city_name,order_id,
                    case 
                        when contains(event_type_agg, 'accepted') = false and  contains(order_status_agg, 'customerCancelled') =true then 'COBRA'
                        when  contains(order_status_agg, 'expired') = true and contains(event_type_agg, 'accepted') = false then 'Mapped_expiry'
                        when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =false and contains(event_type_agg, 'customer_cancelled') =true)  then 'OCARA'
                        when contains(event_type_agg, 'accepted') = true and  (contains(event_type_agg, 'rider_cancelled') =true and contains(event_type_agg, 'customer_cancelled') =false)  then 'OCARA'                                
                        else 'others'
                    end as order_level_end_state
                from
                (
                    select * 
                    from

                    (
                        select yyyymmdd, city_name,order_id,
                        array_agg(event_type) over (partition by order_id) as event_type_agg,
                        array_agg(order_status) over (partition by order_id) as order_status_agg,
                        row_number() over(partition by order_id) as row_num

                        from immutabl_raw

                    )
                    where row_num=1 
                )
                )
                group by 1,2,3 

                ),

    -- to get fm and lm 

    v3 as (

        select order_id, --captain_id,

        --case when fm<=500 then '0_500M'
         --when fm<=1000 then '0.5_to_1km'
         --when fm<=1500 then '1km_1.5km'
         --when fm<=2000 then '1.5_2km'     
         --when fm<=2500 then '2_2.5km' 
        --when fm<=3000 then '2.5_3km' 
        --when fm<=3500 then '3_3.5km' 
        --when fm<=4000 then '3.5_4km' 
        --when fm<=4500 then '4_4.5km'
        --when fm>4500 then '4.5km_above' 
        --end as fm_buckets,

         case when lm<=3000 then '0_3km'
              when lm<=5000 then '3_5km'
              when lm<=8000 then '5_8km'
              when lm<=12000 then '8_12km'
            when lm>12000 then '12km_above'
            end as lm_bucket

        from
            (
                select
                        order_id,
                        --captain_id,
                        avg(captains_firstmiledistance) as fm,
                        avg(last_mile_distance) as lm

                from
                        g2n.flattened_captains
                    where yyyymmdd = cast({dt} as varchar)
                    and service_detail_id in ('5ef2bc5b85846b775f97d170','61769b50d21e19252c6f60e8')

                group by 1--,2
            )
        )



        select yyyymmdd,time_period,cluster,city_name,test_ctrl,segment,lm_bucket, --fulfilment_flag,fm_buckets
        gross_orders,
        net_orders,
        accepted_order,
        rider_busy_pings,
        rider_rejeted_pings,
        accepted_pings,
        all_accepted_pings,
        already_accepted_pings,
        total_pings,
        ocara,
        cobra,
        mapped_expiry,
        (cast(accepted_order as double)/gross_orders*1.00) as AOR,
        (cast(all_accepted_pings as double)/total_pings*1.00) as APR

        from
        (

        select yyyymmdd,time_period,cluster,city_name,test_ctrl,segment,lm_bucket, --fulfilment_flag,fm_buckets
        count(distinct order_id) as gross_orders,
        sum(net_orders) as net_orders,        
        count(distinct case when accepted_pings>=1 then order_id end) as accepted_order,
        sum(riderBusy_pings) as rider_busy_pings,
        sum(riderreject_Pings) as rider_rejeted_pings,
        sum(accepted_pings) as accepted_pings,
        sum(accepted_pings+rider_accept_failure_pings) as all_accepted_pings,
        sum(already_accepted_pings) as already_accepted_pings,
        sum(rider_ack_pings) as total_pings,
        sum(OCARA) as ocara,
        sum(COBRA) as cobra,
        sum(Mapped_expiry) as mapped_expiry

        from
        (
            select t1.yyyymmdd,t1.city_name,segment,lm_bucket,test_ctrl, --fm_buckets,fulfilment_flag,data_riderId
            t1.order_id, pickup_location_hex_8,time_period,
            riderBusy_pings,riderreject_Pings,accepted_pings,
            already_accepted_pings,rider_accept_failure_pings,rider_ack_pings,
            net_orders,COBRA,OCARA,Mapped_expiry,cluster


            from order_filtering as t1

            left join rider_pings as t2

            on t1.order_id = t2.data_orderid 

            left join immutable_v2 as t4
            
            on t2.data_orderid = t4.order_id 
           
           left join v3

           on t2.data_orderid = v3.order_id --and t2.data_riderId = v3.captain_id

       

            left join (select city_name,test_ctrl, hex_id ,time_bucket
                from experiments_internal.auto_apr_fm_reduction_exp_230704 where city_name in ('Pune','Hyderabad')  ) hex_data
                on t1.pickup_location_hex_8=hex_data.hex_id and t1.time_period = hex_data.time_bucket  


           left join            
           (
           select city,service,uid,segment
            from experiments_internal.holdout_areas_v2
            where service='Auto' and city in ('Hyderabad','Pune')

           ) as v1 
           on concat(t1.pickup_location_hex_8,'-',t1.time_period) = v1.uid

          left join(
              Select hex_id, cluster 
               from datasets.city_cluster_hex where resolution = 8 and city in ('Hyderabad','Pune')  
                ) as hxdt
                on t1.pickup_location_hex_8=hxdt.hex_id                
            
            
          where case when t1.yyyymmdd >='20230710' and concat(t1.pickup_location_hex_8,t1.time_period) 
               in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                    '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                    '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                    '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                    '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                    '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                    '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                    '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                    '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                    '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                    '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                    '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                    '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                    '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                    '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                    '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                    '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                    '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                    '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                    '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                    '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                    '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                    '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                    '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                    '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                    '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                    '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                    '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                    '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                    '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                    '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                    '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                    '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                    '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                    '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                    '8860a24a65fffffmorning_peak'

                    ) then 1 else 0 end =0

                and 
                   concat(t1.time_period,t1.pickup_location_hex_8) not in 
                    ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                    'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                    'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                    'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                    'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                    'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                    'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                    'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                    'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                    )
        )

        group by 1,2,3,4,5,6,7 

        )



    """.format(dt=i.strftime('%Y%m%d'))
    
    df_test=pd.read_sql_query(query, metabase_connection)
    
    print(i)
    
    data=data.append(df_test)
    

2023-07-04 00:00:00
2023-07-05 00:00:00
2023-07-06 00:00:00
2023-07-07 00:00:00
2023-07-08 00:00:00
2023-07-09 00:00:00
2023-07-10 00:00:00
2023-07-11 00:00:00
2023-07-12 00:00:00
2023-07-13 00:00:00
2023-07-14 00:00:00
2023-07-15 00:00:00
2023-07-16 00:00:00
2023-07-17 00:00:00


DatabaseError: {'message': 'Encountered too many errors talking to a worker node. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (http://10.16.100.3:8080/v1/task/20230725_050158_00354_pdava.11.4.0/results/0/1 - 248 failures, failure duration 300.44s, total failed request time 310.44s)', 'errorCode': 65540, 'errorName': 'PAGE_TRANSPORT_TIMEOUT', 'errorType': 'INTERNAL_ERROR', 'failureInfo': {'type': 'io.trino.operator.PageTransportTimeoutException', 'message': 'Encountered too many errors talking to a worker node. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (http://10.16.100.3:8080/v1/task/20230725_050158_00354_pdava.11.4.0/results/0/1 - 248 failures, failure duration 300.44s, total failed request time 310.44s)', 'cause': {'type': 'java.io.UncheckedIOException', 'message': 'Server refused connection: http://10.16

In [50]:
data.sort_values(by)

,yyyymmdd,time_period,cluster,city_name,test_ctrl,segment,lm_bucket,gross_orders,net_orders,accepted_order,...,rider_rejeted_pings,accepted_pings,all_accepted_pings,already_accepted_pings,total_pings,ocara,cobra,mapped_expiry,AOR,APR
0,20230704,evening_peak,Madhura Nagar Metro,Hyderabad,ctrl_1,in-high,3_5km,54,16,20,...,514.0,20.0,20.0,90.0,1263.0,3.0,26.0,8.0,0.370370,0.015835
1,20230704,afternoon,Bibewadi Pune,Pune,None,in-low,3_5km,3,2,3,...,44.0,5.0,5.0,24.0,85.0,0.0,0.0,0.0,1.000000,0.058824
2,20230704,afternoon,Langar Houz,Hyderabad,ctrl_3,out-high,3_5km,78,20,33,...,1384.0,35.0,36.0,276.0,3288.0,13.0,26.0,19.0,0.423077,0.010949
3,20230704,evening_peak,Banjara Hills,Hyderabad,ctrl_3,out-high,0_3km,107,7,10,...,376.0,10.0,10.0,23.0,939.0,1.0,76.0,21.0,0.093458,0.01065
4,20230704,morning_peak,Shamshabad,Hyderabad,ctrl_4,out-low,8_12km,6,5,6,...,16.0,7.0,12.0,59.0,87.0,2.0,0.0,0.0,1.000000,0.137931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20870,20230717,rest_morning,Saroor Nagar,Hyderabad,test_2,balanced-low,5_8km,1,1,1,...,0.0,1.0,4.0,9.0,13.0,0.0,0.0,0.0,1.000000,0.307692
20871,20230717,afternoon,Nanakramguda 2,Hyderabad,test_2,balanced-low,3_5km,1,1,1,...,8.0,3.0,3.0,24.0,35.0,1.0,0.0,0.0,1.000000,0.085714
20872,20230717,afternoon,Alwal 2,Hyderabad,ctrl_1,in-high,3_5km,2,0,0,...,43.0,0.0,0.0,0.0,88.0,0.0,1.0,1.0,0.000000,0.0
20873,20230717,rest_evening,Falaknuma,Hyderabad,test_2,out-high,None,9,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,None


In [47]:
data.to_clipboard()

In [ ]:
-- order level characteristics 
